# Loading modules


In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import scipy.ndimage as ndimage 
import skimage 
from scipy import stats 

# Centers Initialization 
PeakFinder uses the histogram of the image to find the initial centers of the grayscales. 


In [ ]:
import cmath 
class PeakFinder:
  def __init__(self, img, mincenters): 
    self.hist, _  = np.histogram(img, bins=256)
    self.s1 = [i for i in range(1,255) if self.hist[i] > max(self.hist[i-1], self.hist[i+1])]
    self.s2 = [i for i in range(1, len(self.s1)-1) if self.hist[self.s1[i]] > max(self.hist[self.s1[i-1]],self.hist[self.s1[i+1]])]
    self.s3 = [x for x in self.s2  if x >= 2.5/100 * self.hist.argmax()]
    lst = []
    i = j = 0
    while j < len(self.s3): 
      while j < len(self.s3) and abs(self.s3[i]- self.s3[j]) < 15:
        j += 1 
      lst.append(self.s3[i:j].copy())
      i = j 
    self.s4 = [max(e) for e in lst]
    set1 = {x for x in self.s4 if cmath.polar(complex(x,self.hist[x]))[1] > cmath.pi/6} 
    set2 = {x for x, y in zip(self.s4, self.s4[1:]) if abs(complex(x,self.hist[x])- complex(y, self.hist[y])) >= 40 }
    self.s5 = sorted(set1|set2)
    self.all = sorted([self.s1, self.s2, self.s3, self.s4, self.s5], key = len)
    for e in self.all:
      if len(e) >= mincenters:
        self.res = e
        self.all.insert(0,e)
        break 



# HGMM

Apply EM with GMM on the histogram of a gray scale image to obtain an initial Segmentation. 



In [ ]:
class EMGMM:
  def __init__(self, priors, means, vars, min_var = 1e-3, var_regularizer = 1e-3, epsilone = 1e-5, 
               max_iterations = 1000):
    self.priors = np.array(priors)
    self.means = np.array(means, dtype = np.float)
    self.vars = np.array(vars, dtype = np.float )
    self.var_regularizer = var_regularizer 
    self.epsilone = epsilone 
    self.max_iterations = max_iterations
    self.min_var = min_var 
  

  def gamma(self):
    res =  np.empty((self.hist.size, self.means.size))
    data = np.arange(256)
    for j in range(self.means.size):
      res[:,j] = self.priors[j] * stats.norm.pdf(data, loc = self.means[j],
                                                              scale = np.sqrt(self.vars[j]))

    return res 
  def llk(self):
    return (np.log(self.gamma().sum(axis = -1)) * self.hist).sum() / self.hist.sum()
  
  def e_step(self):
    self.latent_distr = self.gamma()
    self.latent_distr =  self.latent_distr / self.latent_distr.sum(axis =-1).reshape(-1,1)
    self.expected_size = (self.hist.reshape(-1,1) * self.latent_distr).sum(axis = 0)

  def m_step(self):
    self.priors = self.expected_size / self.hist.sum()
    self.means = self.hist.reshape(-1,1) * np.arange(256).reshape(-1,1) * self.latent_distr
    self.means = self.means.sum(axis = 0) / self.expected_size
    self.vars = self.latent_distr * ((np.arange(256).reshape(-1,1) - self.means)**2) * self.hist.reshape(-1,1)
    self.vars = self.vars.sum(axis = 0) / self.expected_size
    self.vars[self.vars < self.min_var] = self.var_regularizer
  

  def next(self, new_llk):
    self.cur_iter += 1 
    if(abs(self.old_llk - new_llk) < self.epsilone):
      self.stop = True 
      self.sucess = True 
    elif self.cur_iter >= self.max_iterations:
      self.stop = True 
      self.sucess = False 
    
  def fit(self, hist):
    self.hist = hist 
    self.cur_iter = 0 
    new_llk = self.llk()
    self.stop = False 
    while not self.stop: 
      self.old_llk = new_llk
      self.e_step()
      self.m_step()
      new_llk = self.llk()
      self.next(new_llk) 
  
  def predict(self, data):
    lst = []
    #for mu, sigma in zip(self.means, self.vars):
    #  lst.append(stats.norm(data, loc = mu, scale = np.sqrt(sigma) ))
    #a = np.stack(lst)
    a = np.stack([stats.norm.pdf(data, loc = mu, scale= np.sqrt(sigma)) for mu, sigma in zip(self.means, self.vars)])
    return a.argmax(axis = 0)

# Image Iterators
Used To Traverse the image pixels in diverse orders 

## The base Class for ImageIterators


In [ ]:
from collections.abc import Iterable 
class ImageIterator(Iterable):

  def __init__(self, img, lbls = None):
    self._image = img
    self._labels = lbls 

  
  @property
  def shape(self):
    return self._image.shape 
  
  @property
  def size(self):
    return self._image.size 

## Raster ImageIterator 
Traverse the pixels row by row column by colum up-down left-right

In [ ]:
class RasterImageIterator(ImageIterator):
  def __iter__(self):
    h, w  = self.shape 
    for y in range(h):
      for x in range(w):
        yield (y,x)

## Intertwined Image Iterator

Left-right for even rows
Right-left for odd rows  

In [ ]:
class IntertWinedImageIterator(ImageIterator):
  def __iter__(self):
    h, w = self.shape 
    x_start = 0 
    x_end = w
    x_step = 1 
    for y in range(h):
      for x in range(w) if y % 2 == 0 else reversed(range(w)):
        yield (y,x)

## Snail Curve Image Iterator


In [ ]:
class SnailCurveImageIterator(ImageIterator):

  def __iter__(self):
    if 'cache' in self.__dict__:
      yield from self.cache 
    else:
      self.cache = []
      h, w = self.shape  
      start_y = start_x = 0 
      end_y = h - 1
      end_x = w - 1
      while len(self.cache) < w * h:
        y = start_y
        for x in range(start_x, end_x +1):
          self.cache.append((y,x))
          yield (y,x)
        start_y += 1 

        x = end_x 
        for y in range(start_y, end_y + 1):
          self.cache.append((y,x))
          yield (y,x)
        end_x -= 1 

        y = end_y 
        for x in reversed(range(start_x, end_x +1 )):
          self.cache.append((y,x))
          yield (y,x)
        end_y -= 1

        x = start_x 
        for y in reversed(range(start_y, end_y + 1)):
          self.cache.append((y,x))
          yield (y,x)
        start_x += 1 

## ZigZag Image Iterator

In [ ]:
class ZigZagImageIterator(ImageIterator):
  def __iter__(self):
    if 'cache' in self.__dict__:
      yield from self.cache 
    else:
      self.cache = []
      h, w =  self.shape
      line = h; column = w 
      for k in range(line+ column):
        if  k % 2 != 0:
          x = min(k, column - 1)
          while x >= max(0, k- line +1):
            y = k- x 
            self.cache.append((y,x))
            yield (y,x)
            x -= 1 
        else: 
          y = min(k, line -1)
          while y >= max(0, k - column +1):
            x = k - y 
            self.cache.append((y,x))
            yield (y,x)
            y -= 1 

## Mappping Classes 
Maps N-d Points to a scalar using diverse space fillilg curves (ZCurves and HilbertCurves)

In [ ]:
class ZMapping:
  @staticmethod
  def _ZIndex(pt, nbits):
    """
    Computes  the z-index of a multi-dimensional point pt 
    """
    z = 0 
    mask = 1<<nbits -1 
    for i in range(nbits):
      for coord in pt:
        z += coord& mask
        z <<= 1 
      mask >>= 1 
    return z

  @staticmethod
  def _ZPointAt(zidx, dim, nbits):
    """
    Computes the coordinate of a point in the Z curve Given its index 
    """
    pt = [0] * dim
    mask = 1 << (dim * nbits -1)
    for j in range(nbits):
      for i in range(dim):
        pt[i] <<= 1 
        pt[i] += int((zidx & mask)> 0)
        mask >>= 1 
    return tuple(pt)

  def __init__(self, dim, nbits):
    self.dim = dim 
    self.nbits = nbits 
  

  def ZIndex(self, pt):
    """
    Computes the Zindex of a given point of the curve 
    """
    return ZMapping._ZIndex(pt, self.nbits)
  
  def ZPointAt(self, zidx):
    """
    Compute a Point From its Zindex 
    """
    return ZMapping._ZPointAt(zidx, self.dim, self.nbits)

   

In [ ]:
class HilbertMapping:
  @staticmethod
  def HilbertAxes(transposedIndex, bits):
    X = list(transposedIndex)
    n = len(X)
    N = 2 << (bits -1)
    t = X[n-1] >> 1
    for i in reversed(range(1, n)):
      X[i] ^= X[i-1]
    X[9] ^= t 
    Q = 2 
    while Q != N:
      P = Q - 1 
      for i in reversed(range(1, n)):
        if (X[i]&Q) != 0:
          X[0] ^= P
        else:
          t = (X[0]^X[i])&P 
          X[0] ^= t 
          X[i] ^= t
      Q <<= 1
    return tuple(X) 
  
  @staticmethod
  def HilbertIndexTransposed(hilbertAxes, bits):
    
    X = list(hilbertAxes)
    n = len(X)
    M = 1 <<(bits-1)
    Q = M
    while Q > 1:
      P = Q - 1 
      for i in range(n):
        if (X[i]&Q) != 0:
          X[0] ^= P 
        else:
          t = (X[0]^X[i]) & P 
          X[0] ^= t 
          X[i] ^= t 
      Q >>= 1 
    for i in range(1, n):
      X[i] ^= X[i-1]
    
    t = 0 
    Q = M 
    while Q > 1: 
      if (X[n-1]& Q) != 0:
        t ^= Q -1 
      Q >>= 1 
    for i in range(n): 
      X[i] ^= t 
    return tuple(X)

  @staticmethod
  def _HilberIndex(HilbertIndex, bits):
    H = 0 
    Mask = 1 << bits - 1 
    for i in range(bits):
      for j in HilbertIndex:
        H += j& Mask
        H <<= 1 
      Mask >>= 1 
    return H 

  @staticmethod
  def TransposeHilbertIndex(HilbertIndex, bits, dimensions):
    THilbert = [0] * dimensions
    Mask = 1 << (bits * dimensions - 1)
    for j in range(bits):
      for i in range(dimensions):
        THilbert[i] <<= 1 
        THilbert[i] += int(HilbertIndex& Mask)
        Mask >>= 1 
    return tuple(THilbert) 




  def __init__(self, dim, nbits):
    self.dim = dim
    self.nbits  = nbits 

  def HilbertIndex(self, pt):
    t = HilbertMapping.HilbertIndexTransposed(pt, self.nbits)
    return HilbertMapping._HilberIndex(t, self.nbits)

  def HilbertCurveAt(hidx):
    t = HilbertMapping.TransposedHilberIndex(hidx, self.nbits, self.dim)
    return HilbertMapping.HilbertAxes(t, this.nbits)




## Z2D Image iterator
Uses a Zcurve filling the 2D pixel grid to traverse the image

In [ ]:
class Z2DImageIterator(ImageIterator):

  def __init__(self, img, lbls = None):
    super().__init__(img, lbls)
    h, w = self.shape 
    self.cache = [(y,x) for y in range(h) for x in range(w)]
    from math import log2, ceil
    nbits = int(ceil(log2(max(h,w)))) 
    map = ZMapping(2, nbits)
    self.cache.sort(key = map.ZIndex)
  
  def __iter__(self):
    yield from self.cache 

## Hilbert 2D Image Iterator 
Uses a 2D space Hilbert Filling curve to traverse the image

In [ ]:
class Hilbert2DImageIterator(ImageIterator):

  def __init__(self, img, lbls = None):
    super().__init__(img, lbls)
    h, w = self.shape 
    self.cache = [(y,x) for y in range(h) for x in range(w)]
    from math import log2, ceil
    nbits = int(ceil(log2(max(h,w)))) 
    map = HilbertMapping(2, nbits)
    self.cache.sort(key = map.HilbertIndex)
  
  def __iter__(self):
    yield from self.cache 

## Code Image Iterator
Iterates over neighborhood independent pixels

In [ ]:
class CodeImageIterator(ImageIterator):
  def __init__(self, pos, step, img, lbls = None ):
    super().__init__(img, lbls)
    self.y, self.x = pos 
    self.stepy, self.stepx = step 
  
  def __iter__(self):
    h, w = self.shape
    for y in range(self.y, h, self.stepy):
      for x in range(self.x, w, self.stepx):
        yield (y,x)

## Codes Image Iterator 
Splits the image into neighborhood independent subsets 'Codes' and iterates over the codes one by one

In [ ]:
class CodesImageIterator(ImageIterator):
  def __init__(self, steps, imgs, lbls = None):
    super().__init__(imgs, lbls)
    self.stepy, self.stepx = steps

  def __iter__(self):
    for y in range(self.stepy):
      for x in range(self.stepx):
        yield from  CodeImageIterator((y,x), (self.stepy, self.stepx), self._image, self._labels)

## Z3D Image Iterator 
Uses a 3D Z filling curve based on (y,x, gray[y,x]) to traverse the image.

In [ ]:
class Z3DImageIterator(ImageIterator):
  def __init__(self, img, lbls = None):
    super().__init__(img, lbls)
    h, w = self.shape 
    from math import ceil, log2
    nbits = int(ceil(log2(max(h,w,self._image.max()))))
    map = ZMapping(3, nbits)
    b = 2 ** nbits 
    stepx = b // w
    stepy = b//h 
    stepi = b // self._image.max()
    self.cache = [(y,x) for y in range(h) for x in range(w)]
    self.cache.sort(key = lambda p : map.ZIndex((p[0] * stepy,p[1] * stepx, self._image[p] * stepi)))
  
  def __iter__(self):
    yield from self.cache 

## Hilbert 3D Image iterator  
Uses a 3D  Hilbert Space filling curve 

In [ ]:
class Hilbert3DImageIterator(ImageIterator):
  def __init__(self, img, lbls = None):
    super().__init__(img, lbls)
    h, w = self.shape 
    from math import ceil, log2
    nbits = int(ceil(log2(max(h,w,self._image.max()))))
    map = HilbertMapping(3, nbits)
    b = 2 ** nbits 
    stepx = b // w
    stepy = b//h 
    stepi = b // self._image.max()
    self.cache = [(y,x) for y in range(h) for x in range(w)]
    self.cache.sort(key = lambda p : map.HilbertIndex((p[0] * stepy ,p[1] * stepx , self._image[p] * stepi)))
  
  def __iter__(self):
    yield from self.cache 

# Utility functions 

return the 8-immediate neighbors of a pixel

In [ ]:
def get8neighbors(y,x):
  s = {(y+i, x+j) for i in range(-1, 2) for j in range(-1, 2)}
  return s- {(y,x)}

reads an image from a file a ndarray 

In [ ]:
def imread(path):
  img = plt.imread(path)
  if img.ndim > 2:
    img = img[:,:,0]
  if img.dtype is not np.uint8:
    img = img * 255
    img = img.round()
  return img.astype(np.uint8)

# Markov Random Field Model


## Offline MRF Map
Used to descripe MRF Field where the stats are update in offline mode (after the whole image is traversed)


In [ ]:
class OfflineMRFMap:
  def __init__(self, img, lbls, priors, interaction, minvar = 30, var_regularizer = 30):
    assert img.shape == lbls.shape 
    assert lbls.max() + 1 == priors.size == interaction.shape[0] == interaction.shape[1] 
    h, w = img.shape 
    self._minvar = minvar 
    self._varRegularizer = var_regularizer
    self._priors = priors 
    self._interaction = interaction 
    self._image = np.zeros((h+2, w+2), img.dtype)
    self._image[1:-1, 1:-1] = img 
    x = np.zeros((h+2, w+2), lbls.dtype)
    x[1:-1, 1:-1] = lbls 
    self._labelCount = lbls.max() + 1 
    self._next_x = x.copy()
    self.init_stats()
  
  @staticmethod
  def adapt(y,x):
    return (y+1, x+1)
 
  @property 
  def shape(self):
    h, w = self._image.shape 
    return (h-2,w-2)
 
  def __iter__(self):
    h, w = self.shape 
    for y in range(h):
      for x in range(w):
        yield OfflineMRFMap.adapt(y,x)
  
  def init_stats(self):
    self._cur_x = self._next_x.copy()
    self._globalW = 0 
    self._lbl_Hist = np.zeros((self._labelCount,), dtype = np.int)
    self._sums = np.zeros((self._labelCount,), dtype = np.double)
    self._sqSums = self._sums.copy()
    self._localPotentials = np.zeros_like(self._image)
    for y, x in self:
      cur_seg = self._cur_x[y,x]
      cur_gray = self._image[y,x]
      self._lbl_Hist[cur_seg] += 1 
      self._sums[cur_seg] += cur_gray
      self._sqSums[cur_seg] += cur_gray **2 
      self._localPotentials[y,x] = self._priors[cur_seg]
      for ny, nx in get8neighbors(y,x):
        n_seg = self._cur_x[ny, nx]
        self._localPotentials[y,x] += self._interaction[cur_seg, n_seg]
      self._globalW += self._localPotentials[y,x] + self._conditionalLocalW(y,x)
      #logging.info(" globalW = {}".format(self._globalW))
     
  def mean(self, lbl):
    return self._sums[lbl] / self._lbl_Hist[lbl]
  
  def variance(self, lbl):
    sqMean = self.mean(lbl)**2 
    var = self._sqSums[lbl] / self._lbl_Hist[lbl] - sqMean 
    return var if var >= self._minvar else var + self._varRegularizer
 
  def _conditionalLocalW(self, y,x):
    lbl = self._cur_x[y,x]
    sigma = self.variance(lbl)
    mu = self.mean(lbl)
    y = self._image[y,x]
    return (y - mu)**2 /(2* sigma) + np.log(np.sqrt(sigma))
  def conditionalLocalW(self, y, x):
    return self._conditionalLocalW(self, * OfflineMRFMap.adapt(y,x))
 
  def localPriorW(self, y, x):
    y, x = self.adapt(y,x)
    return self._localPotentials[y,x]
 
  def _localPotential(self, y,x, lbl):
    if lbl == self._cur_x[y,x]:
      return self._localPotentials[y,x]
    else:
      lP = self._priors[lbl]
      for ny, nx in get8neighbors(y,x):
        nlbl = self._cur_x[ny, nx]
        lP += self._interaction[lbl, nlbl]
      return lP
 
  def localPotential(self, y, x, lbl):
    y, x = self.adapt(y,x)
    return self._localPotential(y,x, lbl)
 
 
  def localW(self, y, x, lbl):
    y, x = self.adapt(y,x)
    lW = self._localPotential(y,x, lbl)
    cur_lbl = self._cur_x[y,x]
    y = self._image[y,x]
    mu = self.mean(lbl)
    sigma = self.variance(lbl)
    lW += (y - mu)**2 / (2*sigma) + np.log(np.sqrt(sigma))
    return lW
 
  def localFit(self, y, x, lbl):
    return np.exp(-self.localW(y,x, lbl))
  
  def changeLabel(self, y, x, lbl):
    y, x = self.adapt(y,x)
    self._next_x[y,x] = lbl 
  
  def label(self, y, x):
    y, x = self.adapt(y,x)
    return self._cur_x[y,x]
  
  @property
  def globalW(self):
    return self._globalW
 
  @property
  def globalFit(self):
    return np.exp(- self.globalW / (self.size))
  
  @property
  def size(self):
    return (self._image.shape[0] -2) * (self._image.shape[1]-2)
  
  @property
  def labels(self):
    return self._next_x[1:-1, 1:-1].copy()
  
  @property
  def labelCount(self):
    return self._labelCount

## Online MRF Map

Represents an MRF Field where the stats are updated online (after each side labeling)

In [ ]:
class OnlineMRFMap:
 
  def __init__(self, img, lbls, priors, interaction, minvar = 30, var_regularizer = 30):

    assert img.shape == lbls.shape 
    assert lbls.max() + 1 == priors.size == interaction.shape[0] == interaction.shape[1] 
    h, w = img.shape 
    self._minvar = minvar 
    self._varRegularizer = var_regularizer
    self._priors = priors 
    self._interaction = interaction 
    self._image = np.zeros((h+2, w+2), img.dtype)
    self._image[1:-1, 1:-1] = img 
    x = np.zeros((h+2, w+2), lbls.dtype)
    x[1:-1, 1:-1] = lbls 
    self._labelCount = lbls.max() + 1 
    self._x = x
    self.init_stats()
    self._computeGlobalW()
 
  @property
  def globalW(self):
    if self._updateGlobalW:
      self._computeGlobalW()
    return self._globalW
 
 
  @property
  def globalFit(self):
    return np.exp(- self.globalW / (self.size))
 
    
  def label(self, y, x):
    y, x = self.adapt(y,x)
    return self._x[y,x]
 
  def localFit(self, y, x):
    return np.exp(-self.localW(y,x))
 
  def _computeGlobalW(self):
    self._globalW = sum(self._localW(y,x) for y, x in self)
    self._updateGlobalW = False 
    
  
  @staticmethod
  def adapt(y,x):
    return (y+1, x+1)
 
  @property 
  def shape(self):
    h, w = self._image.shape 
    return (h-2,w-2)
 
  def __iter__(self):
    h, w = self.shape 
    for y in range(h):
      for x in range(w):
        yield OnlineMRFMap.adapt(y,x)
  
  def init_stats(self):
    self._lbl_Hist = np.zeros((self._labelCount,), dtype = np.int)
    self._sums = np.zeros((self._labelCount,), dtype = np.double)
    self._sqSums = self._sums.copy()
    for y, x in self:
      cur_seg = self._x[y,x]
      cur_gray = self._image[y,x]
      self._lbl_Hist[cur_seg] += 1 
      self._sums[cur_seg] += cur_gray
      self._sqSums[cur_seg] += cur_gray **2 
      
 
  def _updatestats(self,y, x, oldlbl, newlbl):
    if oldlbl != newlbl:
      self._updateGlobalW = True
      g = self._image[y,x]
      self._lbl_Hist[oldlbl] -= 1 
      self._lbl_Hist[newlbl] += 1 
      self._sums[oldlbl] -= g
      self._sums[newlbl] += g 
      self._sqSums[oldlbl] -= g**2 
      self._sqSums[newlbl] += g**2 
 
 
     
  def mean(self, lbl):
    return self._sums[lbl] / self._lbl_Hist[lbl]
  
  def variance(self, lbl):
    sqMean = self.mean(lbl)**2 
    var = self._sqSums[lbl] / self._lbl_Hist[lbl] - sqMean 
    return var if var >= self._minvar else var + self._varRegularizer
 
 
  def _localLikelihood(self, y, x, lbl):
    oldlbl = self._x[y,x]
    g = self._image[y,x]
    size = self._lbl_Hist[lbl]
    sum = self._sums[lbl]
    sqsum = self._sqSums[lbl]
    if oldlbl != lbl:
      size += 1 
      sum += g 
      sqsum += g**2 
    mu = sum / size 
    sigma = (sqsum / size) - mu **2 
    if sigma < self._minvar:
      sigma += self._varRegularizer
    return (g - mu)**2 / (2 * sigma) + np.log(np.sqrt(sigma))
 
  def localLikelihood(self, y, x, lbl):
    y,x  = self.adapt(y,x)
    return self._localLikelihood(y,x, lbl)
  
  def _conditionalLocalW(self, y,x):
    lbl = self._x[y,x]
    sigma = self.variance(lbl)
    mu = self.mean(lbl)
    g = self._image[y,x]
    r =  (g - mu)**2 /(2* sigma) + np.log(np.sqrt(sigma))
    return r
 
  def conditionalLocalW(self, y, x):
    return self._conditionalLocalW(self, * OnlineMRFMap.adapt(y,x))
 
  def localPriorW(self, y, x):
    y, x = self.adapt(y,x)
    lbl = self._x[y,x]
    return self._localPotential(y,x, lbl)
 
  def _localPotential(self, y,x, lbl):
    lP = self._priors[lbl]
    for ny, nx in get8neighbors(y,x):
      nlbl = self._x[ny, nx]
      lP += self._interaction[lbl, nlbl]
    return lP
 
  def _measureLocalW(self, y, x, lbl):
    return self._localPotential(y,x, lbl) + self._localLikelihood(y, x, lbl)
 
  def measureLocalW(self, y, x, lbl):
    y, x = self.adapt(y,x)
    return self._measureLocalW(y,x, lbl)
 
  def _measureLocalFit(self, y, x, lbl):
    return np.exp(- self._measureLocalW(y,x, lbl))
 
  def measureLocalFit(self, y, x, lbl):
    y, x  = self.adapt(y,x)
    return self._measureLocalFit(y,x, lbl)
   
  def localPotential(self, y, x, lbl):
    y, x = self.adapt(y,x)
    return self._localPotential(y,x, lbl)
 
 
 
 
  def _localW(self, y, x):
    lbl = self._x[y,x]
    lW = self._localPotential(y,x, lbl) + self._conditionalLocalW(y,x)
    return lW
 
  def localW(self, y, x):
    y, x = self.adapt(y,x)
    return self._localW(y,x)
 
 
  
  def changeLabel(self, y, x, lbl):
    y, x = self.adapt(y,x)
    oldlbl = self._x[y,x]
    self._x[y,x] = lbl 
    self._updatestats(y,x, oldlbl, lbl)
  
  @property
  def size(self):
    return (self._image.shape[0] -2) * (self._image.shape[1]-2)
  
  @property
  def labels(self):
    return self._x[1:-1, 1:-1].copy()
  
  @property
  def labelCount(self):
    return self._labelCount
 
  def localFitnessDistr(self, y,x):
    y, x = self.adapt(y,x)
    d = [self._measureLocalFit(y,x,lbl) for lbl in range(self.labelCount)]
    s = sum(d)
    return np.array([v/s for v in d])

# Iterated Conditional Mode 
Greedy algorithm to optimize and MRF model 

## Offline ICM 
The stats are updated after the genration of a whole configuration 


In [ ]:
class OffileICM:
  def __init__(self, img, lbls, priors, interaction, max_iter = 1000, epsilon = 1e-4):
    self._model = OfflineMRFMap(img, lbls, priors, interaction)
    self._current_iteration = 0 
    self._maxIter = max_iter 
    self._iterator = RasterImageIterator(img)
    self._stop = False 
    self._sucess = False 
    self._eps = epsilon


  def best_label(self, y, x):
    ref = float('inf')
    for lbl in range(self._model.labelCount):
      lw = self._model.localW(y,x,lbl)
      if lw < ref:
        ref = lw 
        res = lbl 
    return res 



  def process_pixel(self, y, x):
    lbl = self.best_label(y,x)
    if lbl != self._model.label(y,x):
      self._changed_labels += 1 
      self._model.changeLabel(y,x, lbl)
    
  def next_iteration(self):
    self._lw = self._model.globalW
    self._labels = self._model.labels 
    self._model.init_stats()
    self._current_iteration += 1 
    self.energies.append(self._model.globalW)
    if self._current_iteration > self._maxIter:
      self._stop = True 
    if self._changed_labels <= self._eps or self._model.globalW >= self._lw:
      self._stop = True 
      self._sucess = True 

  def run(self):
    
    self.energies = [self._model.globalW]
    #logging.info(" Started : GW = {}".format(self.energies[-1]))
    while not self._stop:
      self._changed_labels = 0 
      for y, x in self._iterator:
        self.process_pixel(y,x)
      #logging.info(" Iteration {} ".format(self._current_iteration))
      #logging.info(" Changed Labels {}".format(self._changed_labels))
      self.next_iteration()
    #logging.info(" Ended : GW = {}".format(self.energies[-1]))


## OnLine ICM 
The stats are updated after each site relabeling 

In [ ]:
class OnlineICM:

  @property 
  def labels(self):
    return self._model.labels
  
  def __init__(self, img, lbls, priors, interacts, maxiter = 1000, eps =3, iter = None):
    if iter is None:
      iter = RasterImageIterator(img, lbls)
    self._model = OnlineMRFMap(img, lbls, priors, interacts)
    self._image = img 
    self._init_x = lbls 
    self._current_iter = 0 
    self._maxiter = maxiter 
    self._stop = False 
    self._sucess = False 
    self._eps = eps 
    self._iter = iter 
    self.energies = [self._model.globalW]
    self._labels = self._model.labels
    self._lastW = self._model.globalW

    
      
  def _bestLabel(self, y, x):
    w_ref = np.inf 
    for lbl in range(self._model.labelCount):
      w = self._model.measureLocalW(y,x, lbl)
      if w < w_ref:
        w_ref = w 
        lbl_ref = lbl 
    return lbl_ref
      
  def _processSite(self, y, x):
    lbl = self._bestLabel(y,x)
    if lbl != self._model.label(y,x):
      self._changedlabels  += 1 
      self._model.changeLabel(y, x, lbl)
  
  def _nextIteration(self):
    self._current_iter += 1 
    self.energies.append(self._model.globalW)
    if self._current_iter > self._maxiter:
      self._stop = True 
    if self._changedlabels <= self._eps:
      self._stop = True 
    
    if self._lastW < self.energies[-1]:
      self._stop = True 
      self._sucess = True
    else:
      self._lastW = self._model.globalW
      self._lbls = self._model.labels.copy()

  def run(self):
    while not self._stop:
      self._changedlabels = 0 
      for y, x in self._iter: 
        self._processSite(y, x)
      self._nextIteration()
      

# AntColony Optiomizer 
Used ACO to optimize an MRF

## Colony Environment 
Represents the Environment of the colony 
- Pheromone Trails
- Evaporation Rules 
- Parameters of the colony 

In [ ]:
class Environment:

  def __init__(self, img, priors, interacts, tau_0, q0, rho, xi, alpha, beta):
    """
    img = 2d ndarray representing the image pixels 
    priors = 1d ndarray representing the prior weights 
    interacts = 2d ndarray containing the interaction weights 
    tau_0 = (float) initial pheromone qty 
    q0 = stochasticity 
    rho = local evaporation rate 
    xi = global evaporation rate 
    """
    self._tau_0 = tau_0
    self._q0 = q0 
    self._rho = rho 
    self._xi = xi 
    self._priors = priors 
    self._interacts = interacts 
    self._labelsCount = len(priors)
    self._pheromoneTrails = np.full((img.shape + priors.shape), self._tau_0, dtype = np.float64)
    self._image = img.copy()
    self._alpha = alpha 
    self._beta = beta 
    self._hist, _  = np.histogram(img, bins = 256)
    self._centers = PeakFinder(img, self._labelsCount).res 



  def performLocalEvaporation(self, y, x, lbl):
    """
    Applies the local evaporation rule at a specific position on the grid 
    tau_{y,x,lbl} *=  (1 - rho) 
    tau_{y,x,lbl'} = (1- rho) * tau_{y,x, lbl}  +  rho / (#Labels - 1) 
    for all lbl' != lbl 
    """
    self._pheromoneTrails[y,x,:] *= (1-self._rho)
    self._pheromoneTrails[y,x,:] += (self._rho) / (self._labelsCount-1)
    self._pheromoneTrails[y,x,lbl] -= (self._rho) / (self._labelsCount-1)

  def pheromone(y,x,lbl):
    return self._pheromoneTrails[y,x,lbl]

  def performGlobalEvaporation(self, lbls = None):
    """
    Applies the global evaporation rule 
    tau{y,x,l} *= (1 - xi) for all y, x, l 
    tau{y,x,lbl} += xi for all y, x if lbl is in the updat solution 
    """
    self._pheromoneTrails[:,:,:] *= (1- self._xi)
    if lbls is not None:   
      h, w = lbls.shape 
      mask = np.stack([(lbls == i) for i in range(self._priors.size)], axis = -1)
      self._pheromoneTrails[mask] += self._xi 
      #for y in range(h):
      #  for x in range(w):
      #    self._pheromoneTrails[y,x, lbls[y,x]] += self._xi 
      


## Ant 
Each single Ant have its own MRF Map used to generate the heuristic info and uses the pheromone trails for label desirability 

In [ ]:
class Ant:
  def __init__(self, env, queen, iter_cls = None):
    self._env = env 
    self._queen = queen 
    if iter_cls is None:
      self._iter_cls = Z3DImageIterator
    self._init_stats()

  def _init_stats(self):
    import random 
    self._centers = []
    self._priorsWeights = []
    self._means = []
    self._vars = []
    v = (256/self._env._labelsCount)**2 
    ctrs = self._env._centers.copy() 
    for l in range(self._env._labelsCount):
      idx = random.randrange(len(ctrs))
      self._centers.append(ctrs[idx])
      ctrs.pop(idx)
      self._priorsWeights.append(1/self._env._labelsCount)
      self._vars.append(v)
      self._means.append(self._centers[-1])
    self._means.sort()
    img = self._env._image 
    em = EMGMM(self._priorsWeights, self._means, self._vars)
    em.fit(self._env._hist)
    self._lbls = em.predict(img)
    self._map = OnlineMRFMap(img, self._lbls, self._env._priors, self._env._interacts)
    self._iter = self._iter_cls(img, self._lbls)

  def _chooseLabel(self, y, x):
    import random 
    p = random.random()
    h = self._map.localFitnessDistr(y, x)
    weights = self._env._pheromoneTrails[y,x,:] ** self._env._alpha * h ** self._env._beta 
    if p <= self._env._q0:
      #greedy bevahior 
      label = weights.argmax()
    else: 
      #stochastic behavior 
      label, *_ = random.choices(range(self._env._labelsCount), weights)
    self._map.changeLabel(y,x, label)
    self._env.performLocalEvaporation(y,x, label)

  def buildLabeling(self, do_local_search):
    for y,x in self._iter:
      self._chooseLabel(y,x)
    if do_local_search:
      icm = OnlineICM(self._env._image, self._map.labels, self._env._priors, self._env._interacts, self._queen._ls_iter, 0, self._iter)
      icm.run()
      self._map = icm._model
      self._local_done = icm._current_iter
    else:
      self._local_done = 0 

## Queen 
The Queen Manages The Execution of the algorithm 

In [ ]:
from enum import Enum
class UpdateMode(Enum):
  BestSoFar = 0
  IterationBest = 1 
  MixedUpdate = 2 

In [ ]:
class Queen:
  def __init__(self, nb_ants, env, nb_iter, ls_max_iter, do_local_search, update_mode, frequence ):
    self._freq = frequence 
    self._mode = update_mode 
    self._ls_iter = ls_max_iter 
    self._env = env 
    self._ants = [Ant(env, self) for _ in range(nb_ants)]
    self._nb_iter = nb_iter 
    self._nb_ants = nb_ants 
    self._do_ls = do_local_search
    self._bestW = np.inf 
  
  
  def _pickBestLabeling(self):
    energies = [a._map.globalW for a in self._ants]
    local_iters = [a._local_done  for a in self._ants]
    bestW = max(energies)
    bestIdx = energies.index(bestW)
    if self._bestW > bestW:
      self._bestW = bestW
      self._bestLabeling = self._ants[bestIdx]._map.labels 
    if self._mode == UpdateMode.BestSoFar:
      self._env.performGlobalEvaporation(self._bestLabeling)
    elif self._mode == UpdateMode.IterationBest:
      self._env.performGlobalEvaporation(self._ants[bestIdx]._map.labels)
    elif self._nb_iter  % self._freq == 0:
      self._env.performGlobalEvaporation(self._bestLabeling)
    else: 
      self._env.performGlobalEvaporation(self._ants[bestIdx]._map.labels)
    

  
  def _main(self):
    while self._nb_iter != 0:
      for a in self._ants: 
        a.buildLabeling(self._do_ls)
      self._pickBestLabeling()
      self._nb_iter -= 1
    return self._bestLabeling

## The ACO Algorithm 

In [ ]:
class ACOICMMRF:
  def __init__(self, nb_lbl, nb_ants, nb_iter, img, priors, interacts, rho, xi, \
               alpha, beta, q0, do_ls, ls_max_iter, ground = None):
    tau_0 = 1/ nb_lbl
    self._env = Environment(img, priors, interacts, tau_0, q0, rho, xi, alpha, beta)
    self._queen = Queen(nb_ants, self._env, nb_iter, do_ls, ls_max_iter, UpdateMode.BestSoFar, 1)
  
  def run(self):
    self.labels = self._queen._main()



# Evaluation Metrics 

## Unsupervised Metrics

In [ ]:
def Borsotti(img, lbls, ret_info = False, connected = False):
  from skimage.measure import label
  from numpy import sqrt, log
  min_lbl , max_lbl = lbls.min(), lbls.max()
  #lbls += min_lbl +1 # no background pixel split everything 
  #into connected components 
  if not connected:
    cc_lbls  = label(lbls, background=-1)
  else: 
    cc_lbls  = lbls 
  min_lbl, max_lbl = cc_lbls.min(), cc_lbls.max()
  num_lbls = len(set(cc_lbls.flat))
  masks = {v:cc_lbls == v  for v in range(min_lbl, max_lbl+1)}
  areas = {v:masks[v].sum() for v in masks}
  errors = {v:img[masks[v]].std() for v in masks}
  lst_areas = list(areas.values())
  d = {v:lst_areas.count(v) for v in set(lst_areas)} 
  print( {"errors":errors, "areas":areas,  "region/area":d})
  try:
        s =  1/(1000 * img.size) * np.sqrt(num_lbls+1) *\
   sum(errors[i]**2/ (1+ log(areas[i])) + (d[areas[i]]/areas[i])**2 for i in masks) 
  except Exception as e:
    print("error  : ", {"errors":errors, "areas":areas, "masks":masks, "region/area":d})
    print(e)
    
  if not ret_info:
    return s 
  else: 
    return {"metric":s, "errors":errors, "areas":areas, "masks":masks, "region/area":d}

## Supervised Metrics

Preprocess the image to produce a binary filter 

In [ ]:
def postprocess(lbls, gt):
  """
  The groundTruth a binary mask 
  lbls could be a formed by more than 2 labels an arbitrary order, representing 
  a connected regions  extracted from the segmented image.  
  The function returns a binary mask that matches the groudTruth gt as close as possible, 
  by applying the following processing 
  for each label in lbls 
    nb0 = # of pixels in the intersection between the region associated to label and the background (0)
    nb1 = # of pixels in the intersection between the region associated to label and the foreground (1)
    associate label to (0 : background) if nb0 > nb1, otherwise associate label to (1: foreground)  
  """
  from skimage.measure import label 
  lbls = label(lbls, background = -1)
  gt = gt.astype(np.bool)
  output = np.zeros(gt.shape, dtype = np.bool)
  min_lbl, max_lbl = lbls.min(), lbls.max()
  for label in range(min_lbl, max_lbl):
    m = lbls == label
    nb0 = (m&(~gt)).sum() 
    nb1 = (m&gt).sum()
    if nb1 >= nb0:
      output[m] = 1 
    else: 
      output[m] = 0 
  return output  


In [ ]:
def accuracy_metric(gt, output, normalize = True):
  from sklearn.metrics import accuracy_score 
  return accuracy_score(gt.ravel(), output.ravel(), normalize = normalize)

In [ ]:
def dice_metric(gt, output, pos_label = 1, average = "weighted"):
  from sklearn.metrics import f1_score 
  return f1_score(gt.ravel(), output.ravel(), pos_label = pos_label, average = average)

# Benchmark Algorithms 

## WaterShed V1 

In [ ]:
def WaterShed1(img, file = None):
  from skimage.segmentation import watershed 
  from skimage.filters import sobel 
  img = sobel(img)
  labels =  watershed(img)
  if file is not None:
    file.write(str(locals()))
  return labels 

## WaterShed 2
https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_marked_watershed.html

In [ ]:
def WaterShed2(image, file = None):
  from scipy import ndimage as ndi
  from skimage.morphology import disk
  from skimage.segmentation import watershed
  from skimage import data
  from skimage.filters import rank
  from skimage.util import img_as_ubyte
  image = img_as_ubyte(image)
  # denoise image
  denoised = rank.median(image, disk(2))
  # find continuous region (low gradient -
  # where less than 10 for this image) --> markers
  # disk(5) is used here to get a more smooth image
  markers = rank.gradient(denoised, disk(5)) < 10
  markers = ndi.label(markers)[0]
  # local gradient (disk(2) is used to keep edges thin)
  gradient = rank.gradient(denoised, disk(2))
  # process the watershed
  labels = watershed(gradient, markers)
  if file is not None: 
    file.write(str(locals()))
  return labels 

## MeanShift Algorithm

In [ ]:
def MeanShift(img, file = None, sdf = 10):
  from sklearn.cluster import MeanShift
  from skimage.transform import resize  
  ms =   MeanShift()
  h, w  = img.shape 
  simg = resize(img,(h//sdf, w//sdf), preserve_range = True )
  ms.fit(simg.reshape((-1,1)))
  labels =  ms.predict(img.reshape((-1,1))).reshape(img.shape)
  if file is not None:
    file.write(str(locals()) + "\n")
    file.write(str(ms.__dict__) + "\n")
  return labels 

## Normalized GraphCut
https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_ncut.html

In [ ]:
def NormalizedCut(img, factor = 30, file = None):
  from skimage import data, segmentation, color
  from skimage.future import graph
  from matplotlib import pyplot as plt
  from skimage.util import img_as_ubyte 
  start = time.perf_counter()
  img = img_as_ubyte(img)
  #print(img.dtype)
  #img = img.astype(np.double)
  #print(img.min(), img.maaax())
  labels1 = segmentation.slic(img, img.size// factor)
  #out1 = color.label2rgb(labels1, img, kind='avg', bg_label=0)
  #plt.imshow(labels1, "gray")
  #plt.show()
  g = graph.rag_mean_color(img, labels1, mode='similarity')
  labels2 = graph.cut_normalized(labels1, g)
  if file is not None: 
    file.write(str(locals()) + "\n")
  return labels2 

# Segmentation Algorithms as simple function img --> labels  

In [ ]:
def em(img, nb_reg = 2, file = None): 
  em = EMGMM(1/nb_reg * np.ones((nb_reg,)) , np.arange(0,256, 256/nb_reg), 50 *  np.ones((nb_reg)))
  hist, _  = np.histogram(img, bins = 256)
  em.fit(hist)
  if file is not None:
    file.write(str(em.__dict__) + "\n")
  return em.predict(img)


In [ ]:
def sk_em(img, nb_reg = 2, file = None):
  from sklearn.mixture import GaussianMixture 
  gmm = GaussianMixture(n_components=nb_reg)
  gmm.fit(X = img.reshape(-1,1))
  lbls = gmm.predict(img.reshape(-1,1)).reshape(img.shape)
  if file is not None:
    file.write(str(gmm.__dict__) + "\n")
  return lbls

In [ ]:
def kmeans(img, nb_reg = 2, sdf = 10, file = None):
  from sklearn.cluster import KMeans
  from skimage.transform import resize  
  km =   KMeans(n_clusters = nb_reg)
  h, w  = img.shape 
  simg = resize(img,(h//sdf, w//sdf), preserve_range = True )
  km.fit(simg.reshape((-1,1)))
  labels =  km.predict(img.reshape((-1,1))).reshape(img.shape)
  if file is not None:
    file.write(str(locals()) + "\n")
    file.write(str(km.__dict__) + "\n")
  return labels

In [ ]:
def offline_icm(img, nb_reg = 2, priors = None, interaction= None, max_iter = 10, file = None):
  if priors is None: 
    priors = np.zeros((nb_reg,))
  if interaction is None:
    interaction = np.ones((nb_reg, nb_reg))- np.identity(nb_reg)
  lbls = em(img, nb_reg)
  icm = OffileICM(img, lbls , priors, interaction, max_iter = max_iter)
  icm.run()
  if file is not None:
    file.write(str(icm.__dict__) + "\n")
    file.write(str(icm._model.__dict__) + "\n")
  return icm._model.labels 

In [ ]:
def online_icm(img, nb_reg = 2, priors = None, interaction = None, max_iter = 10, iter = None, file = None):
  if priors is None: 
    priors = np.zeros((nb_reg,))
  if interaction is None:
    interaction = np.ones((nb_reg, nb_reg)) - np.identity(nb_reg)
  if iter is None:
    iter = Hilbert3DImageIterator(img)
    lbls = em(img, nb_reg)
  icm = OnlineICM(img, lbls, priors, interaction, iter = iter, maxiter=max_iter)
  icm.run()
  if file is not None:
    file.write(str(icm.__dict__) + "\n")
    file.write(str(icm._model.__dict__) + "\n")
  return icm._model.labels 

In [ ]:
def aco_icm_mrf(img, nb_reg = 2, nb_ants = 2, nb_iter = 10, priors = None, 
                interaction = None, rho = 0.01, xi = 0.05, alpha = 2, beta = 1, 
                q0 = 0.5, perf_local = True, local_iter = 10, file = None):
  if priors is None: 
    priors = np.zeros((nb_reg,))
  if interaction is None:
    interaction = np.ones((nb_reg, nb_reg)) - np.identity(nb_reg)
  aco = ACOICMMRF(nb_reg, nb_ants, nb_iter, img, priors, interaction, rho, xi, alpha, beta , q0, perf_local, local_iter)
  aco.run()
  if file is not None:
    file.write(str(aco.__dict__) + "\n")
    file.write(str(aco._env.__dict__) + "\n")
    file.write(str(aco._queen.__dict__) + "\n")
  return aco.labels 

# Function that takes the number of regions -- > dict where : 
  - each key is an str indicating the algorithm name (used to make the directory of the result) 
  - each value is a function that takes an image and returns 
  the labeling 

In [ ]:
def algs(nb_reg = 2):
  return {
      "HIST_EM" : lambda img, file = None : em(img, nb_reg, file = file),
      "ONLINE_ICM" : lambda img, file = None  : online_icm(img, nb_reg, file = file),
      "OFFLINE_ICM" : lambda img, file = None : offline_icm(img, nb_reg, file = file), 
      "ACO_ICM_MRF" : lambda img, file = None : aco_icm_mrf(img, nb_reg, file = file),
      "EM" : lambda img, file = None : sk_em(img, nb_reg, file = file), 
      "KMEANS" : lambda img, file = None : kmeans(img, nb_reg, file = file)
 
  }

In [ ]:
benchmarks = {"Watershed1" : lambda img, file = None : WaterShed1(img, file = file), 
              "Watershed2": lambda img, file = None : WaterShed2(img, file = file), 
              "MeanShift": lambda img, file = None : MeanShift(img, file = file), 
              "NormalizedGraphCut": lambda img, file = None : NormalizedCut(img, file = file)}

In [ ]:
! unzip /content/Weizmann_Seg_DB_1obj.ZIP

Archive:  /content/Weizmann_Seg_DB_1obj.ZIP
replace 1obj/0677845-r1-067-32_a/human_seg/0677845-r1-067-32_a_7.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import shutil

#Nnshutil.rmtree('/content/1obj')

In [ ]:
import os

In [ ]:
gray_dir = "src_bw"
file_ext = ".png"
params_file = "params.txt"
array_file = "labels.npy"
imgs = []
for spath in glob.glob(r"/content/1obj/*"):
  if os.path.isdir(spath):
    img_name = os.path.basename(spath) + file_ext
    img_path = os.path.join(spath, gray_dir, img_name)
    img = imread(img_path)
    hist, _ = np.histogram(img, bins = 256)
    for dir_name, fct in benchmarks.items():
      dir_path = os.path.join(spath, dir_name)
      os.mkdir(dir_path)
      params_path = os.path.join(dir_path, params_file)
      file = open(params_path, "w")
      start = time.perf_counter()
      labels = fct(img,file)
      end = time.perf_counter()
      file.write(" duration : {}".format(end - start))
      file.close()
      array_path = os.path.join(dir_path, array_file)
      np.save(array_path, labels)
      labels_path  = os.path.join(dir_path, img_name)
      plt.imsave(labels_path, labels, cmap = 'gray')   
    for nb_reg in range(2, 5):
      alg_dict = algs(nb_reg)
      for name, fct in alg_dict.items():
        dir_name = "{}  {}".format(name, nb_reg)
        dir_path = os.path.join(spath, dir_name)
        os.mkdir(dir_path)
        params_path = os.path.join(dir_path, params_file)
        file = open(params_path, "w")
        start = time.perf_counter()
        labels = fct(img,file)
        end = time.perf_counter()
        file.write(" duration : {}".format(end - start))
        file.close()
        array_path = os.path.join(dir_path, array_file)
        np.save(array_path, labels)
        labels_path  = os.path.join(dir_path, img_name)
        plt.imsave(labels_path, labels, cmap = 'gray')   
    

In [ ]:
import os
import glob 
import time 
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm 

def launch_segmentation(fct, img, spath, dir_name, params_file, array_file, img_name):
  dir_path = os.path.join(spath, dir_name)
  params_path = os.path.join(dir_path, params_file)
  array_path = os.path.join(dir_path, array_file)
  labels_path  = os.path.join(dir_path, img_name)
  if os.path.exists(labels_path):
    return 
  elif not os.path.exists(dir_path):
    os.mkdir(dir_path)
  file = open(params_path, "w")
  start = time.perf_counter()
  labels = fct(img,file)
  end = time.perf_counter()
  file.write("\nduration : {}".format(end - start))
  file.close()
  np.save(array_path, labels)
  labels_path  = os.path.join(dir_path, img_name)
  plt.imsave(labels_path, labels, cmap = 'gray')   

In [ ]:
def process_directory(spath, gray_dir, file_ext, params_file, array_file):
  img_name = os.path.basename(spath) + file_ext
  img_path = os.path.join(spath, gray_dir, img_name)
  img = imread(img_path)
  hist, _ = np.histogram(img, bins = 256)
  for dir_name, fct in job_dict.items():
    launch_segmentation(fct, img, spath, dir_name, params_file, array_file, img_name) 


In [ ]:
gray_dir = "src_bw"
file_ext = ".png"
params_file = "params.txt"
array_file = "labels.npy"
job_dict = benchmarks.copy()
for nb_reg in range(2,5):
  job_dict.update({"{}  {}".format(name, nb_reg):fct for name, fct in algs(nb_reg).items()})
inputs = tqdm(spath for spath in glob.glob(r"/content/1obj/*") if os.path.isdir(spath))
p = Parallel(-1)
p(delayed(process_directory)(spath, gray_dir, file_ext, params_file, array_file)\
  for spath in inputs)



0it [00:00, ?it/s]


[]

In [ ]:
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm 

In [ ]:
gray_dir = "src_bw"
file_ext = ".png"
params_file = "params.txt"
array_file = "labels.npy"
imgs = []
job_dict = benchmarks.copy()
for nb_reg in range(2,5):
  job_dict.update({"{}  {}".format(name, nb_reg):fct\
                   for name, fct in algs(nb_reg).items()})
inputs = tqdm(job_dict.items())
for spath in glob.glob(r"/content/1obj/*"):
  if os.path.isdir(spath):
    img_name = os.path.basename(spath) + file_ext
    img_path = os.path.join(spath, gray_dir, img_name)
    img = imread(img_path)
    hist, _ = np.histogram(img, bins = 256)
    p = Parallel(-1)
    p(delayed(launch_segmentation)(fct, img, spath, dir_name, params_file, array_file, img_name) \
              for dir_name, fct in inputs)



  0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
gray_dir = "src_bw"
file_ext = ".png"
params_file = "params.txt"
array_file = "labels.npy"
imgs = []
for spath in glob.glob(r"/content/1obj/*"):
  if os.path.isdir(spath):
    img_name = os.path.basename(spath) + file_ext
    img_path = os.path.join(spath, gray_dir, img_name)
    img = imread(img_path)
    hist, _ = np.histogram(img, bins = 256)
    for dir_name, fct in benchmarks.items():
        launch_segmentation(fct, img, spath, dir_name, params_file, array_file, img_name)
    for nb_reg in range(2, 5):
      alg_dict = algs(nb_reg)
      for name, fct in alg_dict.items():
        dir_name = "{}  {}".format(name, nb_reg)
        launch_segmentation(fct, img, spath, dir_name, params_file, array_file, img_name)

In [ ]:
gray_dir = "src_bw"
gt_dir = "human_seg"
file_ext = ".png"
params_file = "params.txt"
array_file = "labels.npy"
prefixes = {"ACO_ICM_MRF", "HIST_EM", "OFFLINE_ICM", "ONLINE_ICM"}
file_name = "report.txt"
borsotti = "BORSOTTI"
acc = "ACCURACY"
dice = "DICE"
report = {}
for spath in glob.glob(r"1obj/*"):
    if os.path.isdir(spath):
        img_name = os.path.basename(spath)
        img_stats = {}
        report[img_name] = img_stats 
        gt_path = os.path.join(spath, gt_dir)
        gts  = []
        for img_path in glob.glob(gt_path + "/*.png"):
            img = plt.imread(img_path)
            gts.append((img == [1,0,0])[:,:,0])
        for rpath in glob.glob(spath + "/*"):
            if os.path.isdir(spath):
                folder_name = os.path.basename(rpath)
                if any(folder_name.startswith(pre) for pre in prefixes):
                  alg_stats = {}
                  img_stats[folder_name] = alg_stats 
                  labels_path = os.path.join(rpath, array_file)
                  if not os.path.exists(labels_path):
                    continue 
                  labels = np.load(labels_path)
                  alg_stats[borsotti] = Borsotti(img, labels)
                  x = Borsotti(img, labels, connected = False, ret_info=True)
                  print(x)
                  x = input(folder_name)
                  for index, gt in enumerate(gts):
                    lbl = postprocess(labels, gt)
                    acc_key = "{}{}".format(acc, index)
                    dice_key = "{}{}".format(dice, index)
           
                    alg_stats[acc_key] = accuracy_metric(gt, lbl) 

                    alg_stats[dice_key] = dice_metric(gt, lbl)
                if any(folder_name.startswith(pre) for pre in benchmarks):
                  alg_stats = {}
                  img_stats[folder_name] = alg_stats 
                  labels_path = os.path.join(rpath, array_file)
                  if not os.path.exists(labels_path):
                    continue 
                  labels = np.load(labels_path)
                  alg_stats[borsotti] = Borsotti(img, labels, connected= True)
                  x = Borsotti(img, labels, connected = True, ret_info=True)
                  print(x)
                  x = input(folder_name)
                  for index, gt in enumerate(gts):
                    lbl = postprocess(labels, gt)
                    alg_stats[acc + str(index)] = accuracy_metric(gt, lbl) 
                    alg_stats[dice + str(index)] = dice_metric(gt, lbl)

                  

In [ ]:
job_dict = benchmarks.copy()
for nb_reg in range(2,5):
  job_dict.update({"{}  {}".format(name, nb_reg):fct for name, fct in algs(nb_reg).items()})
gray_dir = "src_bw"
gt_dir = "human_seg"
file_ext = ".png"
params_file = "params.txt"
array_file = "labels.npy"
prefixes = set(job_dict) - set(benchmarks)
file_name = "report.txt"
borsotti = "BORSOTTI"
acc = "ACCURACY"
dice = "DICE"
report = {}
for spath in glob.glob(r"1obj/*"):
    if os.path.isdir(spath):
        img_name = os.path.basename(spath)
        img_stats = {}
        report[img_name] = img_stats 
        gt_path = os.path.join(spath, gt_dir)
        gts  = []
        for img_path in glob.glob(gt_path + "/*.png"):
            img = plt.imread(img_path)
            gts.append((img == [1,0,0])[:,:,0])
        for rpath in glob.glob(spath + "/*"):
            if os.path.isdir(spath):
                folder_name = os.path.basename(rpath)
                if any(folder_name.startswith(pre) for pre in prefixes):
                  alg_stats = {}
                  img_stats[folder_name] = alg_stats 
                  labels_path = os.path.join(rpath, array_file)
                  if not os.path.exists(labels_path):
                    continue 
                  labels = np.load(labels_path, allow_pickle=True)
                  alg_stats[borsotti] = Borsotti(img, labels)
                 
                 
                  for index, gt in enumerate(gts):
                    lbl = postprocess(labels, gt)
                    acc_key = "{}{}".format(acc, index)
                    dice_key = "{}{}".format(dice, index)
                    alg_stats[acc_key] = accuracy_metric(gt, lbl) 
                    alg_stats[dice_key] = dice_metric(gt, lbl)
                if any(folder_name.startswith(pre) for pre in benchmarks):
                  alg_stats = {}
                  img_stats[folder_name] = alg_stats 
                  labels_path = os.path.join(rpath, array_file)
                  if not os.path.exists(labels_path):
                    continue 
                  labels = np.load(labels_path, allow_pickle=True)
                  alg_stats[borsotti] = Borsotti(img, labels, connected= True)
                  for index, gt in enumerate(gts):
                    lbl = postprocess(labels, gt)
                    alg_stats[acc + str(index)] = accuracy_metric(gt, lbl) 
                    alg_stats[dice + str(index)] = dice_metric(gt, lbl)

                  

NameError: ignored

In [ ]:
report

{}

In [ ]:
!zip -r /content/res1obj.zip /content/1obj/

	zip warning: name not matched: /content/1obj/

zip error: Nothing to do! (try: zip -r /content/res1obj.zip . -i /content/1obj/)


In [ ]:
from google.colab import files 
files.download("/content/res1obj.zip ")

FileNotFoundError: ignored

In [ ]:
import multiprocessing
from joblib import Parallel, delayed
num_cores = multiprocessing.cpu_count()

In [ ]:
num_cores
import os

In [ ]:
import os
def lauch_segmentation(fct, img, spath, dir_name, params_file, array_file, img_name):
  dir_path = os.path.join(spath, dir_name)
  params_path = os.path.join(dir_path, params_file)
  array_path = os.path.join(dir_path, array_file)
  labels_path  = os.path.join(dir_path, img_name)
  if os.path.exists(labels_path):
    return 
  elif not os.path.exists(dir_path):
    os.mkdir(dir_path)
  file = open(params_path, "w")
  start = time.perf_counter()
  labels = fct(img,file)
  end = time.perf_counter()
  file.write("\nduration : {}".format(end - start))
  file.close()
  np.save(array_path, labels)
  labels_path  = os.path.join(dir_path, img_name)
  plt.imsave(labels_path, labels, cmap = 'gray')   